In [3]:
import numpy as np
import pandas as pd
import time
import random
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [11]:

HIGH_INCOME_THRESHOLD = 70000  # Threshold to categorize high vs low income
CITY_MEDIAN_INCOME = 50000
AFFORDABILITY_RATIO = 0.30  # Max 30% of income on rent


class Household:
    def __init__(self, id, income, amenity_pref, capacity, move_tolerance, stability, cost_sensitivity):
        self.id = id
        self.income = income
        self.budget = AFFORDABILITY_RATIO * income
        self.amenity_pref = amenity_pref      # weight for neighborhood amenity in utility 
        #self.move_tolerance = move_tolerance    # how much benefit required to justify moving
        #self.capacity = capacity
        self.stability = stability           # probability of considering a move
        self.current_unit = None             # reference to current housing unit or location
        self.time_since_move = 0 
        self.cost_sensitivity = cost_sensitivity  # weight for rent cost in utility
        self.current_unit = None

        
    

In [18]:
class HousingUnit:
    def __init__(self, id, neighborhood: 'Neighborhood', rent):
        self.id = id
        self.neighborhood = neighborhood
        self.rent = rent
        self.vacant = True
        self.tenant = None

In [19]:
     def calculate_utility(self, amenity, rent): #Calculate utility with normalized amenity and rent
        normalized_amenity = amenity / 10  # assuming 0-10 scale
        normalized_rent = rent / self.rent_budget  # rent as fraction of budget
        utility = self.amenity_pref * normalized_amenity - self.cost_sensitivity * normalized_rent
        return utility
        

In [20]:
    def move_to(self, unit: HousingUnit):    #Move to a new unit
        unit.vacant = False
        unit.tenant = self
        self.current_unit = unit


In [22]:
    def decide_and_move(self, available_units: list[HousingUnit]):
        if random.random() > (1 - self.stability) or self.time_since_move < 1:     # Only consider moving if stability trigger allows and not in cooldown
            self.time_since_move += 1    # Stability condition not met or still in cooldown
            return

        if self.current_unit is None:
            affordable = [u for u in housing_options 
                         if u.vacant and u.rent <= self.rent_budget]
            if affordable:
                best = max(affordable, 
                          key=lambda u: self.calculate_utility(u.neighborhood.amenity, u.rent))
                self.move_to_unit(best)
            return
        # Calculate current utility with normalized values
        curr_amenity = self.current_unit.neighborhood.amenity
        curr_rent = self.current_unit.rent
        current_utility = self.calculate_utility(curr_amenity, curr_rent)

        best_unit = None
        best_utility = current_utility

        for unit in available_units:
            if unit is self.current_unit or not unit.vacant:
                continue
            if unit.rent > self.budget:
                continue
            new_utility = self.calculate_utility(unit.neighborhood.amenity, unit.rent)
            
            # Only move if significantly better (moving cost = 0.2 utility points)
            if new_utility - 0.2 > best_utility:
                best_utility = new_utility
                best_unit = unit

        if best_unit:
            self.current_unit.vacant = True
            self.current_unit.tenant = None
            self.move_to(best_unit)
            
        
         

In [23]:
class Landlord:
    class Landlord:
        def __init__(self, id, unit: HousingUnit):
            self.id = id
            self.unit = unit
            self.unit.landlord = self
            self.vacancy_count = 0
            self.occupied_count = 0
            self.base_rent = unit.rent  #remember initial rent
            self.price_aggressiveness = random.uniform(0.5, 1.5)  #how reactive to the market
            
    
 


In [ ]:
    def adjust_rent(self, demand_level, neighborhood_avg_rent):
        if self.unit.vacant:
            self.vacancy_count += 1
            self.occupied_count
            
            if self.vacancy_count >= 3:
                reduction = 0.06 * self.price_aggressiveness   #about 3 to 9 percent reduction
                self.unit.rent = self.unit.rent * (1- reduction)
                self.unit.rent = max(self.unit.rent, self.base_rent * 0.75)  # Don't go below 75% of base rent 
        else:
            self.occupied_count += 1
            self.vacancy_count = 0
            
       if demand_level

